# Control The Robot with Joystick

In [1]:
from FR3Py.interfaces import FR3Real
robot = FR3Real()

Interface Running...


Test the interface by reading the robot joints:

In [2]:
robot.get_state()

{'q': array([-4.66836149e-05, -7.85045044e-01,  6.62306962e-05, -2.35659225e+00,
         6.83355540e-05,  1.57074000e+00,  7.85491554e-01,  0.00000000e+00,
         0.00000000e+00]),
 'dq': array([ 9.98833452e-04, -1.74796479e-03,  4.14957034e-04,  8.28514799e-05,
         1.53946317e-03, -1.86469280e-05,  4.01700470e-04,  0.00000000e+00,
         0.00000000e+00]),
 'T': array([ 1.18846558e-01, -4.55847836e+00, -6.82967067e-01,  2.33390026e+01,
         8.24326515e-01,  2.38965559e+00,  9.06659663e-03,  0.00000000e+00,
         0.00000000e+00])}

In [3]:
from pyJoy.joy_daq import JoyManager
import time    
joy1  = JoyManager()
joy1.start_daq(joy_idx=0)
joy1.offset_calibration()
time.sleep(3)

pygame 2.5.1 (SDL 2.28.2, Python 3.8.15)
Hello from the pygame community. https://www.pygame.org/contribute.html
Put your stick at zero location and do not touch it


In [25]:
import time
import numpy as np
from FR3Py.controllers.waypoint_controller import WaypointController
import datetime
current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

controller = WaypointController(kp=4)
# Read the initila pose of the robot
time.sleep(1)
state = robot.get_state()

q, dq = state['q'], state['dq']
p0 = controller.robot.getInfo(q,dq)['P_EE']
R0 = controller.robot.getInfo(q,dq)['R_EE']

T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
start_time = time.time()

x0, y0, z0 = 0.0, 0.0, 0.0 
R0 = np.eye(3)
ef_poses = []
stamps = []
while True:
    trans = controller.robot.getInfo(q,dq)['P_EE']
    rot = controller.robot.getInfo(q,dq)['R_EE']
    stamps.append(time.time())
    pose = np.vstack([np.hstack([rot, trans.reshape(3,1)]), np.array([0,0,0,1])])
    ef_poses.append(pose)
    
    analog, digital = joy1.read_values()
    cmd = np.array([analog[0],analog[1],analog[2]])
    for i in range(3):
        if np.abs(cmd[i])<0.1:
            cmd[i] = 0 

    if digital[0]==0:
        y0 = y0 - cmd[0]*0.2/100
        x0 = x0 - cmd[1]*0.2/100
        z0 = z0 + cmd[2]*0.2/100
    else:
        omega_hat = np.array([[ 0,       -cmd[2],  cmd[1]],
                              [ cmd[2],   0,      -cmd[0]],
                              [-cmd[1],   cmd[0],      0]])
        R0 = R0@(np.eye(3)+omega_hat/100)

    time.sleep(0.01)
    T= T0@np.vstack([np.hstack([R0, np.array([x0,y0,z0]).reshape(3,1)]), np.array([0,0,0,1])])
    state = robot.get_state()
    q, dq = state['q'], state['dq']
    cmd = controller.compute(q,dq, T_cmd=T)
    robot.send_joint_command(cmd[0:7])

KeyboardInterrupt: 

In [26]:
import pickle
ef_poses = np.stack(ef_poses)
stamps = np.array(stamps)
dataset = {'stamp':stamps, 'base_T_ef':ef_poses}
with open(f'nerf_dataset1_{formatted_datetime}.pckl', 'wb') as f:
    pickle.dump(dataset,f)

In [24]:
dataset

{'stamp': array([1.69344500e+09, 1.69344500e+09, 1.69344500e+09, ...,
        1.69344516e+09, 1.69344516e+09, 1.69344516e+09]),
 'base_T_ef': array([[[ 0.99844215, -0.0500552 ,  0.02465269,  0.38426484],
         [-0.04964913, -0.99862512, -0.01681783, -0.05075856],
         [ 0.02546061,  0.01556764, -0.9995546 ,  0.43348248],
         [ 0.        ,  0.        ,  0.        ,  1.        ]],
 
        [[ 0.9984424 , -0.05004603,  0.02466099,  0.38426553],
         [-0.04963985, -0.99862559, -0.01681693, -0.05075778],
         [ 0.02546871,  0.01556657, -0.99955441,  0.43348508],
         [ 0.        ,  0.        ,  0.        ,  1.        ]],
 
        [[ 0.99844199, -0.050056  ,  0.02465755,  0.38426473],
         [-0.04964989, -0.99862511, -0.01681617, -0.05075906],
         [ 0.0254654 ,  0.01556572, -0.99955451,  0.43348342],
         [ 0.        ,  0.        ,  0.        ,  1.        ]],
 
        ...,
 
        [[ 0.81250644, -0.10076659, -0.57417713,  0.51478534],
         [-0.040